# libraries

In [ ]:
!pip install colab-xterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 66.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain_ollama

In [7]:
!pip install langchain langchain_community unsloth[cu118] accelerate bitsandbytes transformers huggingface-hub -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of unsloth[cu118only] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.0 MB/s eta 

# Loading Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading ollama

In [9]:
%load_ext colabxterm
%xterm

The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm


Launching Xterm...

<IPython.core.display.Javascript object>

In [ ]:
%reload_ext colabxterm

# RAG ChatBot

In [ ]:
import json
import numpy as np
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate

# --- Load JSON dataset ---
def load_dataset(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# --- Chunk text + label + id ---
def prepare_chunks(data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)
    chunks = []
    for item in data:
        split_texts = splitter.split_text(item["text"])
        for chunk in split_texts:
            chunks.append({
                "text": chunk,
                "label": item["label"],

            })
    return chunks

# --- Embed all chunks ---
def embed_chunks(embedding_model, chunks):
    texts = [chunk["text"] for chunk in chunks]
    embeddings = embedding_model.embed_documents(texts)
    return np.array(embeddings).astype("float32")

# --- Build FAISS index ---
def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

# --- Retrieve top-k relevant chunks ---
def retrieve_chunks(embedding_model, query, chunks, index, k=3):
    q_emb = np.array([embedding_model.embed_query(query)]).astype("float32")
    distances, indices = index.search(q_emb, k)
    results = []
    max_distance = max(distances[0]) if len(distances[0]) > 0 else 1
    for rank, i in enumerate(indices[0]):
        score = 1 - distances[0][rank] / max_distance if max_distance > 0 else 1
        results.append({
            "chunk": chunks[i],
            "score": round(score * 100, 2)
        })
    return results

# --- QA Prompt Template ---
qa_prompt_template = PromptTemplate(
    template="""
You are an intelligent assistant answering questions in Farsi using ONLY the context below.

Rules:
- You must answer **strictly and only** based on the provided context.
- Do NOT use prior knowledge.
- If the answer is not in the context, respond with: "پاسخ در متن موجود نیست."
-The answer might be in a different phrasing.
-Look for answer carefully
- Answer clearly and accurately, in Farsi only.


Context:
{context}

Question:
{question}

Answer:
""",
    input_variables=["context", "question"]
)

# --- Main QA Loop ---
def main():
    json_path = "/content/drive/MyDrive/Farsi_Sports_Dataset/Final/voting_classifier_small.json"
    print("Loading dataset...")
    data = load_dataset(json_path)

    print("Preparing chunks...")
    chunks = prepare_chunks(data)
    print(f"Total chunks: {len(chunks)}")

    print("Initializing model...")
    llm = ChatOllama(model="llama3.1", temperature=0.3)
    embedding_model = OllamaEmbeddings(model="llama3.1")

    print("Embedding chunks...")
    embeddings = embed_chunks(embedding_model, chunks)

    print("Building FAISS index...")
    index = build_faiss_index(embeddings)

    while True:
        query = input("❓ Ask your question . Type 'exit' to quit: ").strip()
        if query.lower() == "exit":
            print("Goodbye!")
            break

        print("🔎 Retrieving best matching chunks...")
        top_chunks = retrieve_chunks(embedding_model, query, chunks, index, k=5)  # k=5 برای نتایج بهتر

        context = "\n\n".join([c['chunk']['text'] for c in top_chunks])

        prompt = qa_prompt_template.format(context=context, question=query)

        print("🧠 Generating answer...")
        response = llm.invoke(prompt).content

        print("\n📘 Answer:", response)
        print("🔍 Match Info:")
        for i, result in enumerate(top_chunks, 1):
            chunk_meta = result['chunk']
            print(f"\nResult {i}:")
            print(f"🔹 Label: {chunk_meta['label']}")
            print(f"🔹 Score: {result['score']}%")
            print(f"🔹 Matched Text:\n{chunk_meta['text'][:500]}...")
            print("-" * 40)

if __name__ == "__main__":
    main()

Loading dataset...
Preparing chunks...
Total chunks: 100
Initializing model...
Embedding chunks...
Building FAISS index...
❓ Ask your question . Type 'exit' to quit: تیم ملی فوتبال ایران در مسابقات مقدماتی جام جهانی چه نتیجه‌ای گرفت؟
🔎 Retrieving best matching chunks...
🧠 Generating answer...

📘 Answer: شاگردان با نتیجه ۲ بر ۰ برنده شدند.
🔍 Match Info:

Result 1:
🔹 Label: ورزشی
🔹 Score: 2.809999942779541%
🔹 Matched Text:
عنوان: تیم ملی فوتبال ایران در مسابقات مقدماتی جام جهانی به پیروزی رسید.
خلاصه: شاگردان با نتیجه ۲ بر ۰ برنده شدند.
متن: این دیدار حساس در ورزشگاه آزادی برگزار شد و بازیکنان با تلاش فراوان موفق شدند....
----------------------------------------

Result 2:
🔹 Label: ورزشی
🔹 Score: 1.4900000095367432%
🔹 Matched Text:
عنوان: مسابقات تیراندازی با کمان در تهران برگزار شد.
خلاصه: ورزشکاران ملی‌پوش در این مسابقات شرکت کردند.
متن: رقابت‌ها با حضور داوران بین‌المللی برگزار شد....
----------------------------------------

Result 3:
🔹 Label: ورزشی
🔹 Score: 1.4900000095367432%
🔹 Mat

KeyboardInterrupt: Interrupted by user